# Simulação estocastica

## Imports/Libs

In [27]:
from collections import Counter
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import statistics as stats
import scipy.stats as sc
import plotly.graph_objects as go

In [28]:
## Uniform Generator(0,1)

In [29]:
class Uniform01:

    anterior = 2222
    
    def MCL(self, modulo=2**31, a=1103515245, c=12345, semente=None):
        if semente != None:
            self.anterior = semente
        num_aleatorio = (self.anterior * a + c) % modulo
        self.anterior = num_aleatorio
        return num_aleatorio/modulo

# Discrete Distribuitions

## Discrete Uniform distribution

In [30]:
class DiscreteUniform:
    def __init__(self, a=1, b=5):
        self.a = a
        self.b = b
        self.n = b - a + 1
        self.u01 = Uniform01()

    def random(self):
        return math.trunc((self.n + 1) * self.u01.MCL())

    def mean(self):
        return (self.a + self.b) / 2

    def mode(self):
        return 'N/A'

    def median(self):
        return (self.a + self.b) / 2
    
    def variance(self):
        return ((self.n ** 2) - 1) / 12
    
    def skewness(self):
        return 0
    
    def kurtosis(self):
        return (6 * (self.n ** 2) +1) / (5 * (self.n**2)-1)
    
    def q1(self):
        return math.trunc(self.n / 4) - 1 
    
    def q3(self):
        return math.trunc(3 * self.n / 4) - 1

## Poisson distribution

In [31]:
class Poisson:	
	def __init__(self, P=1, lamb=10):
		self.P = P
		self.lamb = lamb
		self.u01 = Uniform01()

	# Random Variable
	def random(self, n=0): 
		while True:
			R = self.u01.MCL()
			self.P = self.P*R
			if self.P < math.e**(-self.lamb):
				return n
			n = n+1
		return -1

	def mean(self):
		return self.lamb

	def median(self):
		return math.floor(self.lamb + 1/3 - 0.02/self.lamb)

	def mode(self):
		return [math.ceil(self.lamb)-1, math.floor(self.lamb)]

	def variance(self):
		return self.lamb

	def skewness(self):
		return self.lamb**(-1/2)

	def kurtosis(self):
		return 3 + 1/self.lamb
	# Falta os Quantiles

# Continuous Distribuitions

## Continuous Uniform Distribuition

In [32]:
from math import sqrt, floor, ceil

class ContinuousUniform:

    def __init__(self, a=1, b=5):
        self.a = a
        self.b = b
        self.u01 = Uniform01()

    def random(self):
        return self.a + (self.b - self.a) * self.u01.MCL()

    def mean(self):
        return (self.a + self.b) / 2

    def mode(self):
        return 'any value in (' + str(self.a) + ', ' + str(self.b) + ')'

    def median(self):
        return (self.a + self.b) / 2
    
    def variance(self):
        return ((self.b - self.a) ** 2) / 12
    
    def skewness(self):
        return 0
    
    def kurtosis(self):
        return 9/5

    def q1(self):
        return (3/4 * self.a) + (1/4 * b)
    
    def q3(self):
        return (1/4 * self.a) + (3/4 * b)

## Gamma Function

In [33]:
def Gamma(n):
    r = 1
    for i in range(2, n):
        r *= i
    return r

## Weibull Distribution

In [34]:
class Weibull:
    def __init__(self, k=1, lamb=1):
        self.k = k
        self.lamb = lamb
        self.u01 = Uniform01()

    def random(self, k=1, lamb=1):
        return self.lamb * ((-math.log(self.u01.MCL()))**(1/self.k))

    def mean(self):
        return self.lamb * Gamma(1 + 1/self.k)

    def median(self):
        return self.lamb * ((math.log(2))**(1/self.k))

    def mode(self):
        if k > 1:
            return self.lamb * (((self.k - 1)/self.k)**(1/self.k))
        return 0
    
    def variance(self):
        return (self.lamb**2) * math.fabs(Gamma(1 + (2/self.k)) - (Gamma(1 + (1     /self.k)) ** 2))

    def skewness(self):
        return (Gamma(1+(3/self.k)) - (3*Gamma(1 + (1/self.k))*Gamma(1 + (2/self.k))) + (2 * (Gamma(1+ (1/self.k))**3))) / (math.fabs(Gamma(1 + (2/self.k)) - (Gamma(1 + (1/self.k))**2)) ** (3/2))

    def kurtosis(self):
        return (Gamma(1+(4/self.k)) - (4*Gamma(1 + (1/self.k))*Gamma(1 + (3/self.k))) + (6 * (Gamma(1+ (1/self.k))**2) * Gamma(1 + (2/self.k))) + (3 * (Gamma(1 + (1/self.k))**4))) / (math.fabs(Gamma(1 + (2/self.k)) - (Gamma(1 + (1/self.k))**2)) ** 2)

    def q1(self):
        return self.lamb * ((math.log(4) - math.log(3)) ** (1/self.k))
    
    def q3(self):
        return self.lamb * (math.log(4) ** (1/self.k))

## Simulating

In [24]:
def Simulate(va, size, name):
    # mean = []
    # mode = []
    # median = []
    # variance = []
    # skewness = []
    # kurtosis = []
    # q1 = []
    # q3 = []
    samples = []
    rows = []
    formated_rows = []

    rlabels = ['Mean', 'Mode', 'Median', 'Variance', 'Skewness', 'Kurtosis', 'Q1', 'Q3']
    formated_rows.append(rlabels)
    for i in range(10):
        sample = []
        row = []
        for j in range(size):
            sample.append(va.random())
        samples.append(sample)

        row.append(stats.mean(sample))
        row.append(stats.mode(sample))
        row.append(stats.median(sample))
        row.append(stats.variance(sample))
        row.append(sc.stats.skew(sample))
        row.append(sc.stats.kurtosis(sample))
        row.append(np.percentile(sample, 0.25))
        row.append(np.percentile(sample, 0.75))
        # print(row)
        
        rows.append(row)
        formated_rows.append(['%.6f' % elem for elem in row])
    
    mean_statistics = []
    for k in range(8):
        aux = []
        for row in rows:
            aux.append(row[k])
        mean_statistics.append(round(stats.mean(aux), 6))
    
    formated_rows.append(mean_statistics)

    clabels = []
    clabels.append('')
    for i in range(10):
        clabels.append(i+1)
    
    clabels.append('Statistic Average')

    fig = go.Figure(data=[go.Table(
        # columnwidth = [80,400],
        header=dict(
            values=clabels,
            # line_color='darkslategray',
            # fill_color='royalblue',
            align=['left','center'],
            # font=dict(color='white', size=12),
        ),
        cells=dict(
            values=formated_rows,
            # line_color='darkslategray',
            # fill=dict(color=['paleturquoise', 'white']),
            # align=['left', 'center'],
            # font_size=12,
            # height=30
        ))
    ])
    fig.update_layout(
        width=1700,
        height=400,
        title_text =  name+', n='+str(size),
    )
    fig.show()

va = ContinuousUniform()
Simulate(va, 1000, 'Continuous Uniform')

va = ContinuousUniform()
Simulate(va, 10000, 'Continuous Uniform')

va = ContinuousUniform()
Simulate(va, 100000, 'Continuous Uniform')

va = Poisson()
Simulate(va, 1000, 'Poisson')

va = Poisson()
Simulate(va, 10000, 'Poisson')

va = Poisson()
Simulate(va, 100000, 'Poisson')

va = DiscreteUniform()
Simulate(va, 1000, 'Discrete Uniform')

va = DiscreteUniform()
Simulate(va, 10000, 'Discrete Uniform')

va = DiscreteUniform()
Simulate(va, 100000, 'Discrete Uniform')

va = Weibull()
Simulate(va, 1000, 'Weibull')

va = Weibull()
Simulate(va, 10000, 'Weibull')

va = Weibull()
Simulate(va, 100000, 'Weibull')
